# Tutorial for SLPs

Credits: Henrique Ennes (https://hlovisiennes.github.io/).

In [1]:
from FHDpy.SLP import SLP

This notebook only deals with the straight line programs (SLP) part of FHDpy and, therefore, is of a lesser interest to the paper associated to this code. Indeed, some of the functions and modules implemented for SLP (e.g., binary transformation and uncompression), are not needed for the experiments involving the computation of the first homology group of 3-manifolds in the other tutotial, but were implemented for the user's convinience. 

SLPs are compressed representations of strings of letters in some monoidal or group (i.e., hard words for assembles of letters with repetition) $\mathcal{L}$. For example, consider the following element of a group generated by 'a','b','c', and 'd'
\begin{equation*}
s = aaaabbbbbddc^{-1}c^{-1}c^{-1}c^{-1}.
\end{equation*}
While it can be naturally written in power-notation form (also known as zipped form)
\begin{equation*}
a^4b^4d^2c^{-4}
\end{equation*}
it can also be written as an SLP

0. a
1. #0.#0
2. #1.#1
3. b
4. #2.#3 
5. #4.#3.#3.#3
6. #5.d.d
7. c
8. #7*
9. #6.#8.#8.#8.#8

Here, each line is called an assignement and their content (right hand side) are called variables, which are either
    - simple: a variable starting with a letter that belongs to the alphabet, including the empty letter ''
        Ex.: 'a', 'b', 'c*',''
    - proper assignment: a product of previous assignment in the SLP of form '#Integer'
        Ex.: '#3'
Each assignment is a either a simple variable, a product of proper assignements, or a product of proper assignements and simple variables. The symbol * is reserved for negations ('A3*' = A3^{-1}, '#3*' = (#3)^{-1}). The symbol '.' is reserved for multiplication and power in the SLP and should not be used in symbols. By recursively susbtituting the assignments, we get back the $\textit{uncompressed}$ word $s$, made of letters in $\mathcal{L}$.

In [2]:
# We usually represent the SLP as the list of assignements, in which case it is said to be in list form.
slp_list_form = [
    'a',
    '#0.#0',
    '#1.#1',
    'b',
    '#2.#3', 
    '#4.#3.#3.#3',
    '#5.d.d',
    'c',
    '#7*',
    '#6.#8.#8.#8.#8']

# This creates an object in the SLP class
slp = SLP(slp_list_form)

# This prints the SLP's assignments in a user-friendly way.
print('Compressed SLP:\n', slp)

# We can also uncompress the SLP.
# In general, it uncompresses all assignments, but in most cases, we only care about the last one.
uncompressed_slp = slp.get_uncompressed(inplace = False).list_form[-1]
print('\n\nUncompressed SLP:', uncompressed_slp)

Compressed SLP:
 0. a
1. #0.#0
2. #1.#1
3. b
4. #2.#3
5. #4.#3.#3.#3
6. #5.d.d
7. c
8. #7*
9. #6.#8.#8.#8.#8


Uncompressed SLP: a.a.a.a.b.b.b.b.d.d.c*.c*.c*.c*


We call the complexity of an SLP the sum of all lengths in the assignements. In some cases, SLPs might give a compressed representation of words and, the more a letter is repeated, the greater is the compression gain (in fact, it is not hard to think of words that can be compressed with exponentially shorter SLPs). We call the size of the SLP, when uncompressed, its length. Notice that we do not need to uncompress the word to get its length, the SLP package does that for us with dynamical programming.


In [3]:
print('Complexity: ', slp.complexity())
print('Length: ', len(slp))
# Notice that each letter in the uncompressed word is separated by a period.
print('Length (uncompressed): ', len(uncompressed_slp.split('.'))) 

Complexity:  22
Length:  14
Length (uncompressed):  14


Just for fun, let us make an SLP whose complexity is (much) better than its length

In [4]:
complexity = 30
slp_very_compressed_list_form = ['x'] + [f'#{i}.#{i}' for i in range(complexity)]
very_compressed_slp = SLP(slp_very_compressed_list_form)
print('Complexity: ', very_compressed_slp.complexity())
print('Length: ', len(very_compressed_slp))

Complexity:  61
Length:  1073741824


You can try uncompressing the above SLP to compute its length, but it will soon realize that your memory might blow up for large values of `complexity`.

There are other nice operations we can do to an SLP. For example, we can count the number of occurances of a symbol of $\mathcal{L}$ when uncompressed, again, without uncompressing anything.

In [5]:
print(slp.count('a'))
print(slp.count('c'))

4
4


Notice that the count method is not sign sensitive (for reasons of being more convenient when working with FHDpy). However, it does have a sign sensitive alternative.

In [6]:
print(slp.signed_count('c'))
print(slp.signed_count('c*'))

0
4


You can get the inverse of the word represented by the SLP (again, with dynamic programming, this is measured on the SLP's complexity) and not on its length.

In [7]:
inverse_slp = slp.inverse(inplace = False)
print('Uncompressed inverse of the SLP: ', inverse_slp.get_uncompressed(inplace = False).list_form[-1])

Uncompressed inverse of the SLP:  c.c.c.c.d*.d*.b*.b*.b*.b*.a*.a*.a*.a*


Or get an arbitrary power $s^k$, using as much compressible as possible.

In [8]:
power =  2
slp_power = slp.get_power(2, inplace = False)
print('s^2: ', slp_power.get_uncompressed().list_form[-1])

s^2:  a.a.a.a.b.b.b.b.d.d.c*.c*.c*.c*.a.a.a.a.b.b.b.b.d.d.c*.c*.c*.c*


We can substitute a letter in $\mathcal{L}$ for any other letter or even SLP.  

In [9]:
c_for_a = slp.substitute('c', 'a', inplace = False)
print('Changing c for a: ', c_for_a.get_uncompressed().list_form[-1])

short_slp = SLP(['a', 'c', '#0*.#1'])
c_for_slp = slp.substitute('c', short_slp, inplace = False)
print('Changing c for another SLP: ', c_for_slp.get_uncompressed().list_form[-1])

Changing c for a:  a.a.a.a.b.b.b.b.d.d.a*.a*.a*.a*
Changing c for another SLP:  a.a.a.a.b.b.b.b.d.d.c*.a.c*.a.c*.a.c*.a


And even delete some variable from the SLP.

In [10]:
print('Deleting a: ', slp.delete('a').get_uncompressed().list_form[-1])
print('Deleting c: ', slp.delete('c').get_uncompressed().list_form[-1])

Deleting a:  b.b.b.b.d.d.c*.c*.c*.c*
Deleting c:  a.a.a.a.b.b.b.b.d.d


Finally, we say that an SLP is in binary (Chomsky normal form) if all assignements are either
    
1. a simple variable
    
2. have at most two proper variables without negations

3. a negation of a single proper assigmenet

We can always transform an SLP into a Chomsky normal form.

In [11]:
print(slp.get_binary())

0. a
1. #0.#0
2. #1.#1
3. b
4. #2.#3
5. #4.#3
6. #5.#3
7. #6.#3
8. d
9. #7.#8
10. #9.#8
11. c
12. #11*
13. #10.#12
14. #13.#12
15. #14.#12
16. #15.#12
